In [1]:
cd ./drive/My\ Drive/Colab\ Notebooks/data

/content/drive/My Drive/Colab Notebooks/data


In [2]:
# !wget --quiet http://www.manythings.org/anki/deu-eng.zip

In [3]:
# !unzip deu-eng.zip

In [4]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import unicodedata
import re
import io
import os
import time

# dataset

In [5]:
filepath = './deu.txt'

In [6]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
    if unicodedata.category(c) != 'Mn')
  
def preprocess(s):
  s = unicode_to_ascii(s.lower().strip())

  s = re.sub(r"([?.!,¿])", r" \1", s)
  s = re.sub(r'[" "]+', " ", s)

  s = re.sub(r"[^a-zA-Z?.!,¿]+", " ", s)
  
  s = s.strip()

  s = '<start> ' + s + ' <end>'
  return s

In [7]:
en_s = u'May I borrow this book?'
print(preprocess(en_s))

<start> may i borrow this book ? <end>


In [8]:
def create_dataset(path, num):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess(w) for w in l.split('\t')[:2]]  for l in lines[:num]]

  return zip(*word_pairs)

In [9]:
eng, deu = create_dataset(filepath, None)
print(eng[-1])
print(deu[-1]) 

<start> doubtless there exists in this world precisely the right woman for any given man to marry and vice versa but when you consider that a human being has the opportunity of being acquainted with only a few hundred people , and out of the few hundred that there are but a dozen or less whom he knows intimately , and out of the dozen , one or two friends at most , it will easily be seen , when we remember the number of millions who inhabit this world , that probably , since the earth was created , the right man has never yet met the right woman . <end>
<start> ohne zweifel findet sich auf dieser welt zu jedem mann genau die richtige ehefrau und umgekehrt wenn man jedoch in betracht zieht , dass ein mensch nur gelegenheit hat , mit ein paar hundert anderen bekannt zu sein , von denen ihm nur ein dutzend oder weniger nahesteht , darunter hochstens ein oder zwei freunde , dann erahnt man eingedenk der millionen einwohner dieser welt leicht , dass seit erschaffung ebenderselben wohl noch 

In [10]:
def tokenize(lang):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  tokenizer.fit_on_texts(lang)

  tensor = tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, tokenizer

In [11]:
def load_dataset(path, num):
  tar_lang, sour_lang = create_dataset(path, num)

  tar_tensor, tar_tokenizer = tokenize(tar_lang)
  sour_tensor, sour_tokenizer = tokenize(sour_lang)

  return tar_tensor, sour_tensor, tar_tokenizer, sour_tokenizer

In [12]:
num = 150000
y_tensor, x_tensor, y_tokenizer, x_tokenizer = load_dataset(filepath, num)

x_max_length, y_max_length = x_tensor.shape[1], y_tensor.shape[1]

In [13]:
print(x_max_length, y_max_length)

24 14


In [14]:
xtr_tensor, xte_tensor, ytr_tensor, yte_tensor = train_test_split(x_tensor, y_tensor, test_size = 0.2)

print(len(xtr_tensor), len(ytr_tensor), len(xte_tensor), len(yte_tensor))


120000 120000 30000 30000


In [15]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [16]:
print ("Input Language; index to word mapping")
convert(x_tokenizer, xtr_tensor[0])
print ()
print ("Target Language; index to word mapping")
convert(y_tokenizer, ytr_tensor[0])

Input Language; index to word mapping
1 ----> <start>
20 ----> wir
75 ----> gehen
9 ----> nicht
53 ----> aus
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
21 ----> we
38 ----> re
43 ----> not
71 ----> going
77 ----> out
3 ----> .
2 ----> <end>


In [17]:
#create tf.dataset
BUFFER_SIZE = len(xtr_tensor)
BATCH_SIZE = 64
steps_per_epoch = len(xtr_tensor)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(x_tokenizer.word_index)+1
vocab_tar_size = len(y_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((xtr_tensor, ytr_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# model

In [18]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences = True,
                                   return_state = True,
                                   recurrent_initializer = 'glorot_uniform')
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [19]:
# encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# # sample input
# example_input_batch, example_target_batch = next(iter(dataset))

# sample_hidden = encoder.initialize_hidden_state()
# sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
# print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

In [20]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()

    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    query_with_time_axis = tf.expand_dims(query, 1) # (batch_size, hidden size)-->(batch_size, 1, hidden size)

    score = self.V(tf.nn.tanh(self.W1(query_with_time_axis)+self.W2(values)))

    attention_weights = tf.nn.softmax(score, axis = 1)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis = 1)

    return context_vector, attention_weights


In [21]:
# attention_layer = BahdanauAttention(10)
# attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

# print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
# print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

In [22]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences = True,
                                   return_state = True,
                                   recurrent_initializer = 'glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.attention = BahdanauAttention(self.dec_units)
  
  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)

    x = self.embedding(x)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis = -1)
    output, state = self.gru(x)
    output = tf.reshape(output, (-1, output.shape[2])) # (batch_size * 1, hidden_size)
    x = self.fc(output)

    return x, state, attention_weights

In [23]:
# decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

# sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
#                                       sample_hidden, sample_output)

# print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

## optimizer and loss function

In [24]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)


In [25]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

# Train and evaluate

In [26]:
import numpy as np
def train_step(inp, targ, enc_hidden, acc_object):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([y_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
      loss += loss_function(targ[:, t], predictions)
      pred = np.argmax(predictions, axis = 1)
      acc_object.update_state(targ[:, t], pred)
      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss, acc_object

In [27]:
EPOCHS = 10

for epoch in range(EPOCHS):
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  acc = tf.keras.metrics.Accuracy()

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss, acc = train_step(inp, targ, enc_hidden, acc)
    total_loss += batch_loss

    if batch % 500 == 0:
      print('Epoch {} Batch {} Loss {:.4f} Acc {:.4f}'.format(epoch + 1, batch, batch_loss.numpy(), acc.result().numpy()))

  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f} Acc {:.4f}\n'.format(epoch + 1, total_loss / steps_per_epoch, acc.result().numpy()))

Epoch 1 Batch 0 Loss 4.8912 Acc 0.0000
Epoch 1 Batch 500 Loss 2.0075 Acc 0.1820
Epoch 1 Batch 1000 Loss 1.5110 Acc 0.2213
Epoch 1 Batch 1500 Loss 1.3718 Acc 0.2492
Epoch 1 Loss 1.6761 Acc 0.2680

Epoch 2 Batch 0 Loss 1.0275 Acc 0.3401
Epoch 2 Batch 500 Loss 0.7942 Acc 0.3776
Epoch 2 Batch 1000 Loss 0.7373 Acc 0.3896
Epoch 2 Batch 1500 Loss 0.6718 Acc 0.3991
Epoch 2 Loss 0.7927 Acc 0.4050

Epoch 3 Batch 0 Loss 0.4443 Acc 0.4447
Epoch 3 Batch 500 Loss 0.4065 Acc 0.4516
Epoch 3 Batch 1000 Loss 0.4753 Acc 0.4528
Epoch 3 Batch 1500 Loss 0.4505 Acc 0.4543
Epoch 3 Loss 0.4724 Acc 0.4554

Epoch 4 Batch 0 Loss 0.2981 Acc 0.5048
Epoch 4 Batch 500 Loss 0.3528 Acc 0.4848
Epoch 4 Batch 1000 Loss 0.3821 Acc 0.4836
Epoch 4 Batch 1500 Loss 0.2831 Acc 0.4823
Epoch 4 Loss 0.3221 Acc 0.4821

Epoch 5 Batch 0 Loss 0.2009 Acc 0.5276
Epoch 5 Batch 500 Loss 0.2274 Acc 0.5056
Epoch 5 Batch 1000 Loss 0.1987 Acc 0.5037
Epoch 5 Batch 1500 Loss 0.3024 Acc 0.5016
Epoch 5 Loss 0.2363 Acc 0.5006

Epoch 6 Batch 0 Loss

In [28]:
def evaluate(sentence):
  # sentence = preprocess(sentence)

  inputs = [x_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=x_max_length, padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([y_tokenizer.word_index['<start>']], 0)

  for t in range(y_max_length):
    predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_out)

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += y_tokenizer.index_word[predicted_id] + ' '

    if y_tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [29]:
from nltk.translate.bleu_score import sentence_bleu

def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  score = sentence_bleu(sentence, result)
  print("BLEU score:", score)

In [30]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [31]:
translate(deu[1000])

Input: <start> setz dich dorthin . <end>
Predicted translation: sit there . <end> 
BLEU score: 0.9036020036098448


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [32]:
translate(deu[2000])

Input: <start> bist du in form ? <end>
Predicted translation: are you fit ? <end> 
BLEU score: 0.9146912192286945


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [37]:
translate(deu[160000])

Input: <start> ich dachte , du wohnst bei deiner familie . <end>
Predicted translation: i thought you were family . <end> 
BLEU score: 0.8529987544592307


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [38]:
translate(deu[200000])

Input: <start> das buch handelt vom leben im vereinten konigreich . <end>
Predicted translation: the book is on the most of the top of the top of the 
BLEU score: 0.674961814926904


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [39]:
translate(deu[210000])

Input: <start> allmahlich dammerte mir , dass ich ihn falsch verstanden hatte . <end>
Predicted translation: i was beginning to get wrong . <end> 
BLEU score: 0.7546487907970829


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
